# Recurrent Neural Networks

## Introduction

Recurrent Neural Networks are one of the most popular Neural Network architecture types due to their importance in processing sequential and time series data. While time-series data usually refers to real valued data with a temporal dimension such as stock market prices and sensor outputs, sequential data is a much broader data type which also includes audio, text, and video. 

We begin with an overview of basic RNNs with a simple example. Over the next few weeks we will build on this with more coverage of langauge modelling and then a look at the more sophisticated topics of LSTM and transformer models for sequence processing after the Easter break. 

Note that some of the notes below are based on the tutorial here:
https://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html

For anyone reading these notes out of sequence, remember that this is a short lecture delivered on the same night as the first in-class quiz. 

 ## Recurrent Neural Networks - Overview 

A recurrent neural network is a neural network in which the output of the hidden state of input $i$ is fed to the hidden state alongside the $i+1^{th}$ input. That way, the output at time step k is not only dependent on the input at time step k, but also on the inputs up to point k. 

We can illustrate the most basic RNN instance below with a hidden neuron which is connected back to itself. 

<!-- rnn.png --> 
<img width="300" src="https://drive.google.com/uc?id=1HqHbFVX-l2XBLskh3VF4IMt8AzUOY5iB"/>


We can 'unfold' such a design in time to get a better sense of what is happening. In the figure below we see the RNN with three inputs applied in sequence. The input t=0 will result in a hidden state which is passed to the output at time t=0 but which is also passed into the hidden state at t=1. Thus at t=1 the output of the hidden neuron is dependent not only on the input, but also on the hidden state variable from time t=0. Similarly at t=2 the output of the hidden neuron is dependent on the input at t=2 but also on the output of the hidden state from t=1 which in turn was dependent on the output of the hidden state at t=0. This shows the power of the model, i.e., the hidden state output at t=2 is not only dependent on the input variable at t=2, but on all the inputs that have been seen to date. 

<!-- rnn2.png --> 
<img width="300" src="https://drive.google.com/uc?id=1Fa10zfSV0fcc3TAO_1hWyaBydO1XzD1h"/>

The example above has just one input and output neuron, thus we only have one target variable and one feature. We can trivially extend this approach to multiple target variables and input features by connecting each hidden neuron to its corresponding neuron in subsequent instances of the network as it is unfolded over time. This is illustrated below. Note that for visual purposes we mark hidden-to-hidden links with a separate color, but that this has no baring on the implementation. 

<!-- rnn5.png --> 
<img width="300" src="https://drive.google.com/uc?id=1FQStfBh1RJ4292Whb6rHRUzH1PNEFtO0"/>

This is a vanilla form of recurrent neural network. In practice the architecture adopted is much more complex than this and allows us a greater degree of control in determining what parts of the state variable is saved or passed on to future time steps. But for the moment let's see how this type of model can be put to work. 

## Recurrent Neural Networks - Model Description 

For our Vanilla RNN, let us consider the most basic form of RNN possible, i.e., a RNN which has a link to a single neuron in an input layer, a single neuron in the output layer, and a state variable that is one neuron wide. Lets also assume that the input to this network, X, is a stream of values $\in R$ and that the output from the network, Y, is a stream of values $\in \{0,1\}$. We will refer to the output of the hidden RNN unit as its state, S, and this state is passed to the logit calculations of the output layer and also to the logit calculations of the hidden layer itself. The architecture for this network is illustrated below: 

<!-- rnn_vanilla.png --> 
<img width="500" src="https://drive.google.com/uc?id=1iPbX79FmcUoVf0WynEkeKZtyRP9GhUUO"/>

Some points to note: 
 * Our input layer is clamped to input values $x \in X$ in the normal way. 
 * Our output layer is in this case a hyperbolic tangent function (softmax is not necessary as we only have one output binary class). 
 * Our hidden unit is recursive in that its output value is fed back and concatenated with the content from the input layer. 
 * Weights and biases are assumed for the hidden and output layers in the normal way. We will return to the dimensionality of these variables later. 
 
Operationally the hidden or recursive unit behaves like a typical hidden layer in that a logit is first computed and from this an activation is calculated. We will assume that the activation value is a hyperbolic tangent function, but this need not be the case. The activation function for our vanilla RNN could be a logistic function or other suitable activation function. 

Our visualization above simplifies the temporal nature of the RNN as it shows the output of the RNN unit being fed directly back in to the same unit. For clarity it is useful to think about the network as having to be unrolled out over time to deal with sequences of inputs. We visualizae this below by extending the network out to a sequence of 3 input units at T=0, T=1, and T=2. 

<!-- rnn_vanilla_unrolled.png --> 
<img width="500" src="https://drive.google.com/uc?id=1iTG9nopDMktmN1-840xahMEcuGU_GzcT"/>

Referring to the above we can see now how the output of the hidden layer at time T is passed as input to the hidden layer at T+1. Specifically the $S_{T}$ is concatenated with the actual input $X_{T+1}$. At time T=1 we don't have a true hidden state from a previous step to concatenate with our input $X_{0}$. In this case we typically use an initial value for S which we set to 0. 

As noted above, the output of the hidden RNN unit is commonly referred to as the hidden state, $S$. The value of S for a given time point $t$ is simply: 

$$S_{t} = tanh( (W * (X \oplus S_{t-1})) + b) $$

where $\oplus$ denotes vector concatenation. Thus we see that for a given point in time that the calculation of the hidden layer is typical of any feedforward layer that we have dealt with to date. The difference is that this value is passed back to the network at time $t+1$ as well as being passed to the logit calculation for the output layer. 

### Hidden State Size
In our figures above we assumed our hidden layer had just one hyperbolic tangent unit. This corresponds to having a hidden state size of 1. Naturally however having only one channel to carry information across time will limit the complexity of temporal features that our model will be able to learn. Therefore, in practice we will want a hidden state that is much wider.

Our equation for calculating $S$ above has no such assumptions about the size of the hidden state, and indeed we will see later our implementations of RNNs are built to accommodate much larger hidden states. The approach taken to this is very simple. Rather than having a single non-linear unit, we make use of a so-called RNN cell which can have an arbitrary state size. Internally the cell is implemented as a collection of neurons, i.e., logit and non-linearity calculations, but the specifics of these internal operations are abstracted for us. 

We can illustrate this approach by looking again at the case where T=1 and assume instead a hidden state size of 4. In this illustration $S_{0}$ is the output state from the $0^{th}$ application of the model, and $S_{1}$ will be fed to the following application. 

<!-- rnn_vanilla_states.png --> 
<img width="500" src="https://drive.google.com/uc?id=1iRDZbrkY82WAAdAEOQ9FUaYReluw2CrA"/>

By convention we won't illustrate each of the neurons within the RNN cell and will instead only illustrate the RNN cell itself. For our purposes here we will use a square to capture such RNN cells as opposed to the ovals we have used to date. As above we split the cell in half to illustrate the point that a logit is first calculated before a non-linearity. 

<!-- rnn_vanilla_unrolled_2.png --> 
<img width="500" src="https://drive.google.com/uc?id=1ijarOspm6Qwtue6VS2o4oBrR1zuUjW2A"/>


### Aside: Unrolling the Vanilla RNN for Training 

Above we unrolled the Vanilla RNN over a number of time steps in order to make its operation more transparent. It turns out however that unrolling the RNN is in fact essential to its use. 

Lets consider first the case of using our Vanilla RNN model where the model parameters (i.e., the weights and bias for the hidden and output layers) have already been set. In this case it should be possible for us to feed an input sequence one unit at a time through an unrolled network and produce a set of valid outputs. We have to be careful to copy our output from time point $t$ to time point $t+1$ for concatenation with our actual input, but otherwise there should not be a problem. 

However for training things are a little bit more complex. At the end of an input sequence we should be able to calculate a total training loss on the sequence. That loss is not just from the error on the final output symbol, but should instead be based on the true versus predicted value for the output Y at each time step. We need to propagate errors back from the last time step through multiple steps. 

One question is how many steps should we propagate errors backwards for? There are two extreme conditions. In the first extreme condition we don't propagate errors backwards in time at all. In this case we will not be able to learn temporal dependencies in our data - which defeats the whole purpose of what we are trying to achieve. In the other extreme we allow errors on the final symbol to propagate all the way back, so that in theory corrections to weights at the first time step could be influenced by an error in the final symbol. While this might be appealing in that it would allow arbitrarily long dependencies to be identified, in practice it does not work for two reasons. The first is a purely resource based concern. The further back we allow error propagation, the more complex our model and the more resources (time and memory) that are consumed. The second issue however is more problematic and is an instantiation of the vanishing gradient problem. Essentially over a long time step the repeated multiplication of error gradients will result in the gradient approaching either 0 or $\infty$. 

The solution then in practice is to limit the propagation of errors to a fixed number of time steps. This is referred to as truncated backpropagation. 

### Multiple RNN Layers

In the example above we used a single layer of LSTM units. In practice though we can use multiple layers where the output of one layer feeds into the next layer. This can be visualized as follows: 

<!-- rnn_multi_layer.png --> 
<img width="500" src="https://drive.google.com/uc?id=1iLEQqFBbKfFvXfPMnYFtwI2Ss6E61onL"/>




## Usage Scenario 1: Alarm Tripping

To illustrate the model above, we will create a Trip Alarm for Negative Inputs. This alarm will produce as output the value 0 so long as the input is positive. However if ever the input value turns negative the output of the alarm will switch to output 1 and remain at 1 regardless of the inputs. 

### Step 1 - Data Creation 
We can easily generate some sample data that corresponds to our usage scenario. For training we will limit all our examples to being constant length (currently 20 units below in the code). We will also set the input data to be in the range -1 to 20. We also generate the Target data straightforwardly by iterating over our dataset and applying a function that checks for the occurrence of negative values, and if found sets the output to 1 from the point at which the negative value is found, until the end of the sequence. We will create a large training data set and a smaller test data set with the same methods. 

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# set this below to "./" if you just want to use the current directory to save temporary model files
# else set it to whereever makes sense for you
my_temp_folder = "/Users/rob/temp/"

# Set up some variables used in data collection and in training 
num_training_examples = 10000
num_test_examples = 100
example_length = 20
lower_bound = -1
upper_bound = 20 

# Create Training and Test input variables 
X_train = np.random.randint(low=lower_bound,high=upper_bound, 
                      size=(num_training_examples,example_length)).astype(float)
X_test = np.random.randint(low=lower_bound,high=upper_bound, 
                      size=(num_test_examples,example_length)).astype(float)

# Create Training and Test target variable 
def scan_example(x):
    """Function Used to Create Individual Target Vectors by Scanning Input Vectors"""
    y = np.zeros(shape=np.shape(x))
    for index,val in np.ndenumerate(x):
        if val < 0:
            y[index[0]:] = 1
            break 
    return y
Y_train = np.apply_along_axis(scan_example,1,X_train)
Y_test = np.apply_along_axis(scan_example,1,X_test)

# For processing with Tensorflor / Keras, we expand the dimensionality of our data by one. 
# In other words our inputs will be sets of sets of individual values -- rather than sets of arrays
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
Y_train = np.expand_dims(Y_train, axis=2)
Y_test = np.expand_dims(Y_test, axis=2)

# Print examples from the test data (we add the T to transpose -- this just means the printing is neater. Remove it if you want to see what happens withut it. )
print(X_test[0].T)
print(Y_test[0].T)

[[-1. 10. 13.  4.  9.  0.  3.  8. 11.  9. 17. 12.  1. 14. 13. 10. 16. 15.
  16.  8.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


### Step 2: Model Creation

We will build our recurrent model for this 'alarm system' using a Keras wrapper for Tensorflow. 

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

Next, lets lust define our batch size etc. We will use a state_size of 8. This is already pretty small, but you can still get results if you reduce it to 4. It just might take longer to train. 

In [3]:
batch_size = 100
state_size = 8
num_classes = 1

Next, let's define our model. We will be able to build our model based around the 'Sequential' style constructor that Keras uses. Keep in mind that not ever model that people build is sequential. For now though, it is perfect. 

In [4]:
model = tf.keras.Sequential()

Next, let's define our RNN layer. We need to specify the hidden state size. We also ask Tensorflow not just to output values for every step in the sequence -- not just the final step. Finally, to help guide tensorflow we give it some guidence on the input data dimensions it should expect; here 20 is the length of each example and 1 is the dimensionality of each input element, i.e., just a single number. We don't need to specify the batch size, Keras assumes we will have data in batches -- even if they are just batches of length 1. 

In [5]:
# Input will be passed to an basic RNN layer - sequences will be generated 
model.add(layers.SimpleRNN(state_size,return_sequences=True,input_shape=(20,1)))

Next, we are going to let the full output of the recurrent network be passed to a single output unit. The output unit will be logistic since we are dealing with a single simple classification tasks. 

In [6]:
# add an output layer consisting of 1 single logistic classifier 
model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.sigmoid))

Next, we need to compile our model. We use cross entropy since we want our targets are 0s and 1s and our final layer neuron is pretty good at predicting them. We will use the adam optimiser, and ask for an accuracy metric to be calculated. 

In [7]:
model.compile(loss="binary_crossentropy", 
                   optimizer='adam', 
                   metrics=['accuracy'])

Let's have a look at the built, but not yet trained model. 

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 20, 8)             80        
_________________________________________________________________
dense (Dense)                (None, 20, 1)             9         
Total params: 89
Trainable params: 89
Non-trainable params: 0
_________________________________________________________________


We can see that in the great scheme of things this is a very small model. We could in practice make it much smaller by reducing the size of the hidden state. If we did this our model would likely still converge to a good model, but it would not do so as frequently. We might have to rerun training to get a good model.  

Next, time to train the model. We provide input and output data, and specify a batch size for training as well as the number of epochs. If the model didn't train for you, increase the number of epochs. 

In [9]:
hist = model.fit(X_train, Y_train, 
           batch_size=batch_size,
           epochs=100, verbose=0) 

Note that I had verbosity turned off to save on some printing space, but feel free to set verbose=1 to see the actual progress of training. 

Next, let's predict based on some test data and see what our outputs look like

In [10]:
res = model.predict(X_test,batch_size=batch_size)
print(X_test[0].T)
print(res[0].T)

[[-1. 10. 13.  4.  9.  0.  3.  8. 11.  9. 17. 12.  1. 14. 13. 10. 16. 15.
  16.  8.]]
[[0.9989911  0.99710387 0.99895847 0.9995105  0.9995426  0.9999844
  0.999571   0.9995516  0.99954504 0.99955195 0.99951243 0.9995388
  0.99991584 0.9995473  0.99953914 0.9995485  0.99951935 0.9995233
  0.9995166  0.9995529 ]]


See that in the input data that when there is a -1 that the output switches from values less than 0.5 to values greater than 0.5. In the ideal case this should be a clean separation between 0s and 1s, but typically we need a more powerful model to pull that off. 

## Example 2 - Language Modeling

In the example above we looked at a complete numeric example. While the problem itself was not very sophisticated, we did see a complete recurrent solution. It is easy enough to extrapolate this problem to other domains such as for example real valued outputs. 

For the next example we will move away from simple time series data and look instead at a different sequential data type, namely text. The specific problem we will look at is Language Modeling, or in other words, learning what makes a language a language so that we can predict the next word given a sequence of words. 

This example will be based in part on the Peen Treebank tutorial from the TensorFlow website. However this has been significantly simplified in comparison to the original tutorial. A suggestion is to first work through and make sure you understand the example below. Then move on to the full Penn Treebank example. There are a number of important optimizations in that full tutorial - including variable learning rate - that are worth understanding. 


### Introduction to Language Modeling

Language Modeling is useful to illustrate RNN use as it is a straightforward task that has well understood non-neural implementations, but yet is extremely important in a range of real-world applications and is now commonly implemented with a neural architecture. 

A language model attempts to learn the properties of a language such that when given for example n tokens, the model can accurately predict the $n+1^{th}$ token. For example, given the single token "Merry" the following word is highly likely to be "Christmas". Here just a 1 word context window is enough to help make a good prediction. Normally some more context is required. Given the string "turned the", we would have many reasonable candidates for the next word, but likely none that dominate. However given some more context, such as "she jumped into the car and turned the", a language model might reasonable expect the word "keys" or "key" to be predicted with relatively high probability, while words like "wheel" "engine", or "ignition" might also be given a relatively high probability. 

Linguistic context clues can of course be much further back in the text. Consider the example: "He sat with the ring in his pocket, perspiring, anxious, waiting for the perfect moment that would never come. Finally he slid his hand from his pocket, and asked Marie if she would". Perhaps "marry" would be a suitable next word given this longer context. But looking back only two, three, or even four words would make this unlikely. 

Until recently a language model was built statistically using ngram methods. An algorithm (the estimator) would parse a large body of text and build up tables of all 1-grams, 2-grams, 3-grams and perhaps 4-grams and 5-grams, and use some relatively straightforward probability calculations to give probabilities for individual words given a context.

This statistical form of language modeling is an important tool in Natural Language Processing tasks. Python NLTK (natural langauge toolkit) has a simple implementation of an NGram Language Modeler that you could use to estimate your own language models from texts. Let's use it to parse some simple texts and then use it to make predictions. 

Note that we will be making use of the non neural tool nltk (Natural Langauge Toolkit) to help set up this example. 

Note that you might need to upgrade the version of nltk installed on your notebook. To install the upgrade, just remove the comment below, run the block, and restart the runtime. 

In [11]:
!pip install nltk==3.5

import nltk
from nltk.util import ngrams
from nltk.lm import MLE
from nltk import word_tokenize
# we need to download a special component that is used by the tokenizer below -- don't worry about it. 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Language modeling is all about processing text. So let's go ahead and define some sentences that we will be working with. I'm writing them explicitly here so that you can look at them, but normally we would be importing sentence content from some corpus of sentences defined in a text file or some other external record. The first three examples are defined with newlines between them, but then I leave out the newlines to save on space. 

In [12]:
docs = ["The Lord of the Rings", 
        "The Fellowship of the Ring",
        "The Two Towers",
        "The return of the King",
       "In the Second Age of Middle-earth, the lords of Elves, Dwarves, and Men are given Rings of Power. Unbeknownst to them, the Dark Lord Sauron forges the One Ring in Mount Doom, installing into it a great part of his power to dominate the other Rings, so he might conquer Middle-earth. A final alliance of men and elves battles Sauron's forces in Mordor, where Prince Isildur of Gondor severs Sauron's finger, and the Ring with it, thereby destroying his physical form. With Sauron's first defeat, the Third Age of Middle-earth begins. Unfortunately, the Ring's influence corrupts Isildur, who takes it for himself. Isildur is later killed by Orcs, and the Ring is lost for 2,500 years, until it is found by Gollum, who owns it for five centuries. The Ring is then found by a hobbit named Bilbo Baggins, who turns invisible when he puts it on, but is unaware of its history.",
        "Sixty years later, Bilbo celebrates his 111th birthday in the Shire, reuniting with his old friend, Gandalf the Grey. Bilbo reveals that he intends to leave the Shire for one last adventure, and he leaves his inheritance, including the Ring, to his nephew, Frodo. Although Bilbo has begun to become corrupted by the Ring and tries to keep it for himself, Gandalf intervenes. Gandalf, suspicious of the Ring, tells Frodo to keep it secret and to keep it safe. Gandalf then investigates the Ring, discovers its true nature, and returns to warn Frodo. Gandalf also learns that Gollum was captured and tortured by Orcs, and that Gollum uttered two words during his interrogation: Shire and Baggins. Gandalf instructs Frodo to leave the Shire, accompanied by his friend Samwise Gamgee. Gandalf rides to Isengard to meet with fellow wizard Saruman the White, but learns that he has joined forces with Sauron, who has dispatched his nine undead Nazgûl servants to find Frodo. After a brief battle, Saruman imprisons Gandalf.",
       "Frodo and Sam are joined by fellow hobbits Merry and Pippin, and they evade the Nazgûl, arriving in Bree, where they are meant to meet Gandalf. However, Gandalf never arrives, and they are instead aided by a ranger named Strider, a friend of Gandalf's, who promises to escort them to Rivendell. The hobbits are ambushed by the Nazgûl on Weathertop, and their leader, the Witch-King, stabs Frodo with a cursed Morgul blade. Arwen, an elf and Strider's betrothed, locates Aragorn and the Hobbits and aids Frodo, rescuing him and incapacitating the Nazgûl. She takes him to Rivendell, where he is healed. Frodo meets Gandalf, who escaped Isengard with help from Gwaihir, a giant eagle. That night, Aragorn reunites with Arwen in Rivendell, where they confirm their love and commitment for each other. Arwen's father, Lord Elrond, holds a council that decides the Ring must be destroyed in Mount Doom. While the members argue, Frodo volunteers to take the Ring, accompanied by Gandalf, Sam, Merry, Pippin, elf Legolas, dwarf Gimli, Boromir of Gondor, and Strider, who is revealed to be Aragorn, Isildur's heir and the rightful King of Gondor. Bilbo gives Frodo his sword, Sting. The Fellowship of the Ring sets off, but Saruman's magic forces them to travel through the Mines of Moria, much to Gandalf's displeasure.",
       "The Fellowship discovers that the dwarves within Moria have been slain, and they are attacked by Orcs and a cave troll. They defeat them, but are confronted by Durin's Bane, a Balrog residing within the mines. Gandalf casts the Balrog into a vast chasm, but it drags Gandalf down into the darkness with it. The rest of the Fellowship, now led by Aragorn, reaches Lothlórien, home to elves Galadriel and Celeborn. Galadriel privately informs Frodo that only he can complete the quest, and that one of his friends will try to take the Ring. Meanwhile, Saruman creates an army of Uruk-hai to track down and kill the Fellowship.",
       "The Fellowship leaves Lothlórien by river to Parth Galen. Frodo wanders off and is confronted by Boromir, who tries to take the Ring in desperation. Afraid of the Ring corrupting his friends, Frodo decides to travel to Mordor alone. The Fellowship is then ambushed by the Uruk-hai. Merry and Pippin are taken captive, and Boromir is mortally wounded by the Uruk chieftain, Lurtz. Aragorn arrives and slays Lurtz, and watches Boromir die peacefully. Sam follows Frodo, accompanying him to keep his promise to Gandalf to protect Frodo, while Aragorn, Legolas, and Gimli go to rescue Merry and Pippin.",
       "Awakening from a dream of Gandalf the Grey battling the Balrog, Frodo Baggins and his friend Samwise Gamgee find themselves lost in the Emyn Muil near Mordor and soon become aware that they are being stalked by Gollum, the former owner of the One Ring. After capturing him, a sympathetic Frodo decides to use Gollum as a guide to Mordor, despite Sam's objections.",
       "Meanwhile, Aragorn, Legolas and Gimli pursue the Uruk-hai to save their companions Merry and Pippin. The Uruk-hai are ambushed by a group of Rohirrim, while the two Hobbits escape into Fangorn Forest and encounter Treebeard, an Ent. Aragorn's group later meets the Rohirrim and their leader Éomer, who reveals that they have been exiled by their king Théoden who is being manipulated by Saruman and his servant Gríma Wormtongue into turning a blind eye to Saruman's forces running rampant in Rohan. While searching for the Hobbits in Fangorn, Aragorn's group encounters Gandalf, who, after succumbing to his injuries while killing the Balrog in Moria, has been resurrected as Gandalf the White to help save Middle-earth.",
       "Aragorn's group travels to Rohan's capital city Edoras, where Gandalf releases Théoden from Saruman's influence and Wormtongue is banished. After learning of Saruman's plans to wipe out Rohan with his Uruk-hai army, Théoden decides to evacuate his citizens to Helm's Deep, an ancient fortress that has provided refuge to Rohan's people in times past, while Gandalf departs to acquire the aid of Éomer's army. Aragorn establishes a friendship with Théoden's niece, Éowyn, who quickly becomes infatuated with him. That night, Aragorn experiences a dream of him meeting with Arwen who then tells him that he must depart with Frodo and reassures him when he doesn't feel confident enough that his path is set for him. When the refugees come under attack by Warg-riding Orcs sent by Saruman, Aragorn falls off a cliff and is presumed dead. However, he is awoken by his horse Brego and rides to Helm's Deep. That same night, Arwen is told by her father Elrond that Aragorn will not be returning. He tells her that if she remains in Middle-Earth, only death and destruction await her, even if Aragorn were to survive and become King of Gondor, and if Sauron is defeated again, she will die alone. This convinces her to reluctantly depart for Valinor. The defenders at Helm's Deep are joined by a detachment of Elves from Lothlórien. The Uruk-hai army arrives at Helm's Deep that night and a night-long battle ensues. The Uruk-hai breach the outer wall using gunpowder-like explosives and force the remaining defenders to retreat into the inner castle.",
       "Merry and Pippin, having convinced Treebeard that they were allies, are brought to an Ent Council in Fangorn where the Ents decide not to assist in the war. Pippin then tells Treebeard to take them in the direction of Isengard, where they witness the devastation caused to the forest by Saruman's war efforts. An enraged Treebeard summons the Ents and they storm Isengard, drowning the orcs by breaking their river dam and stranding Saruman in Orthanc.",
       "At Helm's Deep, Aragorn convinces a despairing Théoden to ride out and meet the Uruks in one last charge. Gandalf and Éomer's horsemen arrive at sunrise, turning the tide of the battle. The Uruk-hai flee into Fangorn forest, which has moved closer to the battle at the urging of Treebeard, where they are destroyed. Gandalf warns that Sauron's retaliation will be terrible and swift.",
       "Meanwhile, Gollum leads Frodo and Sam through the Dead Marshes to the Black Gate but convinces them to enter Mordor by an alternative route. Frodo and Sam are captured by the Rangers of Ithilien led by Faramir, brother of the late Boromir. Frodo helps Faramir catch Gollum to save him from being killed and Faramir learns of the One Ring and takes his captives with him to Gondor to win his father's respect. While passing through the besieged Gondorian city of Osgiliath, Sam reveals that Boromir was driven mad by the Ring and tried to take it. An attacking Nazgûl nearly captures Frodo, who momentarily attacks Sam before coming to his senses, forcing Sam to remind him that they are fighting for the good still left in Middle-earth. Faramir is impressed by Frodo and releases them along with Gollum. While leading the hobbits once more, Gollum decides to take revenge on Frodo and reclaim the Ring by leading the group to Her upon arriving at Cirith Ungol."
       "Two Hobbits, Sméagol and Déagol, are fishing when Déagol discovers the One Ring in the river. Sméagol is ensnared by the Ring, and kills his friend for it. He retreats into the Misty Mountains as the Ring twists his body and mind, until he becomes the creature Gollum.",
       "Centuries later, during the War of the Ring, Gandalf leads Aragorn, Legolas, Gimli, and King Théoden to Isengard, where they reunite with Merry and Pippin. Gandalf retrieves the defeated Saruman's palantír. Pippin later looks into the seeing-stone and is telepathically attacked by Sauron. From Pippin's description of his visions, Gandalf surmises that Sauron will attack Gondor's capital Minas Tirith. He rides there to warn Gondor's steward Denethor, taking Pippin with him.",
       "Gollum leads Frodo Baggins and Samwise Gamgee to Minas Morgul, where they watch the Witch-king, leader of the nine Nazgûl, lead an army of Orcs towards Gondor. The hobbits begin climbing a stair carved in the cliff face that will take them into Mordor via a secret way, unaware that Gollum plans to kill them and take the Ring. The Witch-king and his forces strike and overwhelm Osgiliath, forcing Faramir and his garrison to retreat to Minas Tirith.",
       "Gollum disposes of the Hobbits' food, blaming Sam. Believing that Sam desires the Ring, Frodo tells him to go home before he and Gollum continue to the tunnel leading to Mordor. Gollum tricks him into venturing into the lair of the giant spider Shelob. Frodo narrowly escapes and confronts Gollum, telling him that he must destroy the Ring for both their sakes. Gollum attacks Frodo but falls down a chasm. Frodo continues on, but Shelob discovers, paralyses, and binds him. However, Sam returns and injures Shelob, driving her away. Sam hides as Orcs appear and take Frodo with them. The Orcs start a fight because of ownership of Frodo's mithril vest, allowing Sam to escape with Frodo and continue their journey.",
       "Aragorn learns from Elrond that Arwen's strength is fading, having refused to leave and wishing to stay in Middle Earth, reunite with Aragorn, start a family, and willingly become mortal, after seeing a vision of her son with Aragorn. Arwen convinces a reluctant Elrond to accept her choice of staying in Middle Earth and to arm Aragorn with Andúril, a sword recreated by Rivendell blacksmiths on Elrond's orders from the shards of Isildur's sword, Narsil, so he can reclaim his birthright while gaining reinforcements from the Dead Men of Dunharrow. Joined by Legolas and Gimli, Aragorn travels to the Paths of the Dead, recruiting the Army of the Dead by pledging to release them from the curse Isildur put on them.",
       "Faramir is gravely wounded after a futile effort to retake Osgiliath; believing his son to be dead, Denethor falls into madness. Gandalf is left to defend the city against the Orc army, led by Gothmog. As Gothmog's army forces its way into the city, Denethor attempts to kill himself and Faramir on a pyre. Pippin alerts Gandalf and they save Faramir, but a burning Denethor leaps to his death from the top of Minas Tirith just before Théoden and his nephew, Éomer, arrive with the Rohirrim. During the ensuing battle, they are overwhelmed by the Oliphaunt-riding Haradrim, while the Witch-King mortally wounds Théoden. Though Théoden's niece Éowyn destroys the Witch-king with Merry's help, Théoden succumbs to his wounds. Aragorn arrives with the Army of the Dead, who overcome Sauron's forces and win the battle; Aragorn then frees them from the curse.",
       "Aragorn decides to march upon the Black Gate as a distraction so Frodo and Sam can reach Mount Doom. Aragorn's army draws out Sauron's remaining forces and empties Mordor, allowing Frodo and Sam to reach the volcano, but Gollum attacks them just as they reach Mount Doom. As he stands on the ledge over the volcanic fire, Frodo succumbs to the Ring and claims it as his own, putting it on his finger. Despite Frodo being invisible, Gollum manages to find and attack him, biting his finger off to reclaim the Ring. Frodo fights back, and as they struggle over the Ring, both fall off the ledge. Gollum falls into the lava with the Ring and dies. Frodo clings to the side of the ledge, and Sam rescues him as the Ring disintegrates in the lava. As Frodo and Sam escape, Sauron is destroyed along with his forces and the Nine as Mordor crumbles.",
       "Gandalf flies in with eagles to rescue the Hobbits, who awaken in Minas Tirith and are reunited with the surviving Fellowship. Aragorn is crowned King of Gondor and takes Arwen as his queen. The Hobbits return home to the Shire, where Sam marries Rosie Cotton. A few years later, Frodo departs Middle-earth for the Undying Lands with his uncle Bilbo, Gandalf, and the Elves. He leaves Sam the Red Book of Westmarch, which details their adventures. Sam returns to the Shire, where he embraces Rosie and their children."]


Next, to make the text easier to process, we split each sentence into an array of individual words. I print out the first for illustration. 

In [13]:
texts = []
for s in docs:
    texts.append(word_tokenize(s))
print(texts[0])

['The', 'Lord', 'of', 'the', 'Rings']


Traditional language modeling was all about counting statistics. The most important statistics are the number of times individual words occur, but also the number of time particular sequences of words occurred. The sequences here would be small, typically sequences of length 2, 3, 4 or more. These sequences are referred to as n-grams, where 2-grams refer to sequences of 2 words, 3-grams refer to sequences of 3 words and so forth. We use 1-grams to denote sequences of length 1 words -- which are of course just individual words. NLTK has lots of tools for pulling out these sequences for us. For example, lets ask NLTK to produce all 1-grams, 2-grams, and 3-grams from the first sentence. 

In [14]:
print(list(ngrams(texts[0], n=1)))
print(list(ngrams(texts[0], n=2)))
print(list(ngrams(texts[0], n=3)))

[('The',), ('Lord',), ('of',), ('the',), ('Rings',)]
[('The', 'Lord'), ('Lord', 'of'), ('of', 'the'), ('the', 'Rings')]
[('The', 'Lord', 'of'), ('Lord', 'of', 'the'), ('of', 'the', 'Rings')]


These are raw n-grams from the raw text. Often in practice we want to let words at the start and the end of sentences appear in more n-grams than they would in this pure sense. What we want to do is pad the sentences just like we discussed image padding for convolutional neural networks. NLTK can take care of the padding for us. It also does a little bit of extra work for us that isn't really important since our real focus will be on neural implementations. The key point though is that 'train' and 'vocab' below corespond to sets of n-grams and lists of pure words that we have been thinking about so far. 

In [15]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(3, texts)

So far all we have are the lists of n-grams and the vocab list for the complete text. Now we need to do the actual counting of our n-grams in such a way that we can easily generate news sentences and ask questions like 'what are the odds of this sentence having been seen? We do all of this through a model fitting function. How this works is beyond our scope. 

In [16]:
model = MLE(3) 
model.fit(train, vocab)

We can do many things with our trained model, but at its core it is just a tool for collecting probabilities. 

How many times is the word 'the' found in the corpus:

In [17]:
print(model.counts['the'])
print(model.counts['are'])
print(model.counts['Balrog'])

145
18
4


The same phrases, but expressed now in terms of likelihoods over the whole corpus

In [18]:
print(model.score('the'))
print(model.score('are'))
print(model.score('Balrog'))

0.05238439306358381
0.006502890173410405
0.001445086705202312


And how many times is the phrase 'they are' seen? 

In [19]:
print(model.counts[['they']]['are'])

7


And what is the likelihood of seeing 'are' after 'they' rather than seeing 'are' after any other context? 

In [20]:
print(model.score('are','they'.split()))

0.3888888888888889


But what about the more interesting stuff. We can use the model to generate new sentences. We ask for a sentence of a specific length and seed the generator -- re-running with the same seed will result in the same sentence. 


In [21]:
word_list = model.generate(15, random_seed=2)
print(' '.join(word for word in word_list))

vast chasm , but it drags Gandalf down into the lair of the One Ring


Try re-running this with different random seeds -- this just results in different sentences. Then try re-running based on a model that is trained without 3-grams and then without 1-grams. You should see that the model is less fluid. 

Why do you think that is?

###  Perplexity
Language Modeling is not very well served by traditional evaluation metrics such as Precision and Recall. Instead we usually evaluate Language Model performance with a measure called perplexity. Perplexity is essentially a very simple measure with a very grand name. Perplexity is a length normalized inverse probability of a sentence given a specific language model. In other words, given a language model $M$ and a test sentence "W=w_{0},w_{1},..,w_{n}$, the Perplexity of W given by:

$$ PP_{m}(W) = \sqrt[n]{\frac{1}{P(w_{0},w_{1},...,w_{n})}} $$

The intuition of Perplexity is that it gives us an estimate of how much deviation there is between our language model and our test sentence W. If a given sentence W has a higher perplexity with model A than model B, this means that model A is a worse predictor of our sentence. Hence model B is interpreted as being better as it gave a higher probability of our test sentence. 

Therefore when building language models we tend to see smaller Perplexity measures as better and we attempt to reduce the average Perplexity over our data as we train our model. 

Fortunately for the sake of our calculations there is a way to establish a direct relationship between Perplexity and the Cross Entropy loss function we usually use for evaluating our trained models. If we recall, the Cross Entropy is defined as:

$$ H(W) = - \frac{1}{N} ln(P(W)) * P(W) $$

In the case of a language that is assumed to not change over time, it is possible to simplify the definition of entropy. This is a significant simplification, but one that is commonly accepted: 

$$ H(W) = - \frac{1}{N}ln(P(W)) $$

and the perplxity is given by:

$$ PP_{m}(W) = 2^{H(W)} $$

It is worth keeping in mind that this definition is independent of how exactly we operationalise the Cross Entropy function. Therefore even if we use a definition of cross entropy that was used in our loss functions, we can still use it to calculate perplexity as per the equation above. 



### Reading Data
For the Penn Treebank example we will use the same data loading and arrangement operations as are used in the original TensorFlow tutorial in order to keep the overall shape of our modified LSTM code broadly similar to the original. 

First we introduce a number of functions that can be used for loading raw data into a set of integer encoded files. 

## Neural Language Modeling 

Turning from statistical to neural approaches, how might we attempt to achieve a language model with a neural network? One potential model for this design would be to have an input which somehow captures the individual words in some window, and then have an output that captures the relative likelihood of all words in our dictionary. Such a naive model might look like this:

<!-- lmff.png --> 
<img width="500" src="https://drive.google.com/uc?id=1bvTpnJvBN1eXyXXPBSbxwp07rP7ATc-p"/>

(Figure sourced without permission from https://media.licdn.com/mpr/mpr/shrinknp_800_800/AAEAAQAAAAAAAAh8AAAAJGQ5MDRmM2EwLTJkOTMtNDc3ZC05MDAwLWVlYmVmMWE2NmM4Yw.png )

where 3 words are provided as context and a softmax over all words in our dictionary is used in the output layer to decide which work is most likely given an individual context of 3 words. 

This obvious approach can be found to provide some interesting results, and a variant on this model was explored by Bengio et al as far back as 2003. While this feedforward based design can produce useful results given enough training data and training time, the feedforward approach is limited in that it can't take deeper context into account. A recurrent neural networks on the other hand is not limited to taking a fixed history into account. Instead a much deeper history can be explored by allowing the state of the hidden layer to depend not only on an input that represents a word just seen, but also have the hidden state depend on a hidden state further back in time. We will introduce this recurrent model in the next section before returning to language modeling later in the notes. 

In [22]:
import tensorflow as tf

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

1122304/1115394 [==============================] - 0s 0us/step


In [23]:
#text = ""
#for doc in docs:
#    text = text + doc + "\n"

Check some characteristics of the text 

In [24]:
# Take a look at the first 60 characters in text
print(text[:60])

First Citizen:
Before we proceed any further, hear me speak.


We have downloaded the full shakespeare text file. Rather than always working with 100% of the text file, it can be useful just to use a portion of it. The content below picks out a certain proportion of the text to work with. Change the first line to 1.0 if you want to work with all the text. 

In [25]:
extract = 0.5
text = text[:int(extract*len(text))]

Build a new encoding for the text. 

In [26]:
# exract an ordered vocabulary - this will be letters, some numbers etc. 
vocab = sorted(set(text))

# Create mappings between vocab and numeric indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Map all the training text over to a numeric representation
text_as_int = np.array([char2idx[c] for c in text])

Create batches from the complete number encoded dataset. We will be splitting the dataset into sequences of length swq_length. We won't have overlap between these extracts. Let's set up the two variables we need to govern this. 

In [27]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

Let's create a TensorFlow dataset object and ask it to split the dataset into our batches of our specified length.

In [28]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now comes the important parts. Let's start thinking about the training. What we will do is learn to predict one character at a time based on the current character. In other words, given character 'i' we will be predicting character 'i+1'. However what is more than that, we will predict character 'i+1' in the context of having maybe already done this task for X other characters. We will have a state that moves from one character prediction task to the next. 

This way our prediction task pays a lot of attention to the character that is next on our processing list, but it also pays attention to what we have done previously. 

To facilitate this we need to be able to split our dataset sequences on the fly into appropriate encodings for input and target data. We do that with the function below which is applied as a mapping to our input data. 

In [29]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

It is worth visualizing this to make sure we are clear on what is going on:

In [30]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
Step    0
  input: 16 ('F')
  expected output: 45 ('i')
Step    1
  input: 45 ('i')
  expected output: 54 ('r')
Step    2
  input: 54 ('r')
  expected output: 55 ('s')
Step    3
  input: 55 ('s')
  expected output: 56 ('t')
Step    4
  input: 56 ('t')
  expected output: 1 (' ')


To finalise the construction of our dataset, we need to set a batch size and buffer that tensorflow will use to work with the dataset. 

In [31]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Rather than mapping words directly into our LSTM we will map them through a special encoding layer called an embedding. We will return to that next week, but for now we just note the embedding size below: 

In [32]:
embedding_dim = 256

We also need to define the state size for our recurrent layer 

In [33]:
rnn_units = 2048

Now we can construct our model. As noted, the first real layer of our network will be an embedding layer which captures the meaning of our words in a real valued space. 

In [34]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(vocab), embedding_dim,
                          batch_input_shape=[BATCH_SIZE, None]))

Next we define our recurrent layer. This is the bit that will remember state from one classification task to the next. 

In [35]:
model.add(layers.SimpleRNN(rnn_units,return_sequences=True))

Next we define our output layer. We have a final layer where each neuron corresponds to an element in our vocab, i.e., a letter or other character that was found in the complete training data set. Note that this is going to use a linear activation function by default. This is ok, as our cost function is set up to deal with linear activations, i.e., logits, rather than the actual outputs of a softmax function. 

In [36]:
model.add(tf.keras.layers.Dense(len(vocab)))

Our model is built, now let's have a look at it. There should be no surprises here, but it is reassuring to look at. 

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16128     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (64, None, 2048)          4720640   
_________________________________________________________________
dense_1 (Dense)              (64, None, 63)            129087    
Total params: 4,865,855
Trainable params: 4,865,855
Non-trainable params: 0
_________________________________________________________________


Next we need to define our loss function. We will just go with a cross entropy as we might expect for a classification type task. We also compile our model and assume the adam optimiser as always. 

In [38]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In this example that we are borrowing from, the model is saved to a checkpoint as we go along. We will see that the main reason for doing this here is so that we can later run single examples through the model rather than having to run whole batches. 

For the moment we just need to setup where our temporary checkpoint is going to be saved and create a callback object that will be supplied to the keras fit function. You can find out more about saving your model as you go along here:
https://www.tensorflow.org/tutorials/keras/save_and_load

In [39]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = my_temp_folder+'training_checkpoints'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True, 
    save_best_only=True)

directory not used yet.


Everything is setup now. We can kick off training. 

In [40]:
history = model.fit(dataset, epochs=100,callbacks=[checkpoint_callback],verbose=1)
print("training complete.")

Epoch 1/100
86/86 [==============================] - 9s 96ms/step - loss: 3.9909
Epoch 2/100
86/86 [==============================] - 9s 96ms/step - loss: 3.2925
Epoch 3/100
86/86 [==============================] - 9s 96ms/step - loss: 2.7001
Epoch 4/100
86/86 [==============================] - 9s 99ms/step - loss: 2.4048
Epoch 5/100
86/86 [==============================] - 9s 97ms/step - loss: 2.2732
Epoch 6/100
86/86 [==============================] - 9s 98ms/step - loss: 2.1703
Epoch 7/100
86/86 [==============================] - 9s 97ms/step - loss: 2.1094
Epoch 8/100
86/86 [==============================] - 9s 99ms/step - loss: 2.0339
Epoch 9/100
86/86 [==============================] - 9s 100ms/step - loss: 1.9681
Epoch 10/100
86/86 [==============================] - 9s 100ms/step - loss: 1.9161
Epoch 11/100
86/86 [==============================] - 9s 101ms/step - loss: 1.8663
Epoch 12/100
86/86 [==============================] - 9s 98ms/step - loss: 1.8356
Epoch 13/100
86/86 [==

Training is now complete - and in principle we could directly use the model now to pass through a complete batch of input data and get the outputs. In practice though it can be handy to reload the model assume a batch input size of 1 only. This makes it easier for us to run one example at a time through the network. 

Therefore we first recreate our model -- but this time fixing the input layer batch size to one: 

In [41]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(vocab), embedding_dim,
                          batch_input_shape=[1, None]))
model.add(layers.SimpleRNN(rnn_units,return_sequences=True))
model.add(tf.keras.layers.Dense(len(vocab)))

We then need to load in our saved weights to this new model definition and complete construction manually by calling the build function. The build function needs to be called in this case as normally we would get the fit function to do this type of job for us. We won't be calling fit though, as we already have the training complete. 

In [42]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

The whole point of building the language model was so we could generate some new text. So first, let's define a function that can be used to generate such text. The function takes that trained language model as input and a chunk of text that is used to 'prime' the generator. In other words this parameter will be the first few characters of our generated text. 

In [43]:
def generate_text(model, start_string):

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    text_ids_gen = []
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_ids_gen.append(predicted_id)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Put the trained model to use. 

In [44]:
print(generate_text(model, start_string=u"The "))

The d t s
Andeld GHETee t, ire inthen deys athud.
COMusthuit r kis anked, cor fure lirrin urst sppld ser


```
The rtave jun y s n, the:
Thu, d dy l madar l thes,
we!
AULI:
THers t,
Anoe mat'st'd h n chengheananok n
```


OK, so that looks a little bit like English, but it didn't work out that well in practice. 

One of the problems is that our model is good at learning the likelihood of characters given the recent data stream, but it isn't that good at learning to pay attention to further back information in complex ways. This problem is related to the challenge of backpropogation in general, but addressing it is an issue for another week. 


